# Create a Slice, Apply performance tuning and Run iPerf3

This notebook creates a slice on a single site with 3 nodes with each node connected to a NIC_Basic and running on a different host. 
All the nodes are connected via local L2 Network. The notebook then depicts an example on how to apply CPU Pinning and Numa Tuning.
In this notebook, Virtual CPUs for each of the Nodes are pinned to the Physical CPUs residing on the same Numa node as the NIC_Basic connected to the node. In addition, Memory for the Node is also tuned to the same Numa node as the NIC_Basic connected to the node.


## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

## Create the Experiment Slice

The following creates three nodes with basic NICs running on different hosts connected to an isolated local Ethernet.  


In [ ]:
# find two available sites

# we will use NIC_Basic to generate traffic so need sites that have >= 3 workers for this example.
hosts_column_name = 'hosts'

# find two sites with >= 3 hosts
site1 = fablib.get_random_site(filter_function=lambda x: x[hosts_column_name] > 0)
print(f"Site: {site1}")

In [ ]:
slice_name = 'iPerf3-tuned-basic'

node1_name='Node1'
node2_name='Node2'

network_name='net1'
nic_name = 'nic1'
model_name = 'NIC_Basic'

In [ ]:
# Define a subnet to use for the L2 Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[1:]

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

net1 = slice.add_l2network(name=network_name, subnet=subnet)


hosts = [f"{site1.lower()}-w1", f"{site1.lower()}-w2", f"{site1.lower()}-w3"]

for h in hosts:

    # Node1
    node1 = slice.add_node(name=f"{h}-a", cores=16, ram=32, site=site1, image='docker_rocky_8', host=f"{h}.fabric-testbed.net")

    iface1 = node1.add_component(model=model_name, name=nic_name).get_interfaces()[0]
    iface1.set_mode('auto')
    net1.add_interface(iface1)

    node1.add_post_boot_upload_directory('node_tools','.')
    node1.add_post_boot_execute('sudo node_tools/host_tune.sh')
    node1.add_post_boot_execute('node_tools/enable_docker.sh {{ _self_.image }} ')
    node1.add_post_boot_execute('docker pull fabrictestbed/slice-vm-rocky8-multitool:0.0.2 ')

#Submit Slice Request
slice.submit();

## Run iPerf3 without any performance tuning

Run iperf3 between every pair of the Nodes before applying performance tuning.

In [ ]:
slice = fablib.get_slice(slice_name)


source_name = f"{site1.lower()}-w1-a"
source_node = slice.get_node(name=source_name)

source_addr = source_node.get_interface(network_name=network_name).get_ip_addr()

node_names = [f"{site1.lower()}-w1-a", f"{site1.lower()}-w2-a", f"{site1.lower()}-w3-a"]

for i in range(len(node_names)):
    source_node = slice.get_node(name=node_names[i])
    source_addr = source_node.get_interface(network_name=network_name).get_ip_addr()
    
    for j in range(len(node_names)):
        if i==j:
            continue
        print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
        stdout1, stderr1 = source_node.execute("docker run -d --rm "
                                            "--network host "
                                            "fabrictestbed/slice-vm-rocky8-multitool:0.0.2 "
                                            "iperf3 -s -1"
                                            , quiet=True, output_file=f"{source_node.get_name()}-no-tune.log");

        print(f"Source:  {node_names[i]} to Dest: {node_names[j]}")
        dest_node = slice.get_node(name=node_names[j])

        stdout2, stderr2 = dest_node.execute("docker run --rm "
                                            "--network host "
                                            "fabrictestbed/slice-vm-rocky8-multitool:0.0.2 "
                                            f"iperf3 -c {source_addr} -P 10 -t 30 -i 10 -O 10"
                                            , quiet=False, output_file=f"{dest_node.get_name()}-no-tune.log");
        print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

## Perform CPU Pinning and Numa Tuning

In [ ]:
slice = fablib.get_slice(slice_name)

for node in slice.get_nodes():
    # Pin all vCPUs for VM to same Numa node as the component
    node.pin_cpu(component_name=nic_name)
    
    # User can also pass in the range of the vCPUs to be pinned 
    #node.pin_cpu(component_name=nic_name, cpu_range_to_pin="0-3")
    
    # Pin memmory for VM to same Numa node as the components
    node.numa_tune()
    
    # Reboot the VM
    node.os_reboot()

## Re-configure Network Interfaces and Enable Docker post reboot

In [ ]:
slice = fablib.get_slice(slice_name)

# Wait for the SSH Connectivity to be back
slice.wait_ssh()

# Re-configuring the Network
for node in slice.get_nodes():
    node.config()
    stdout1, stderr1 = node.execute("sudo systemctl enable docker")
    stdout1, stderr1 = node.execute("sudo systemctl start docker")    
    stdout1, stderr1 = node.execute("sudo docker pull fabrictestbed/slice-vm-rocky8-multitool:0.0.2 ")
    stdout1, stderr1 = node.execute("sudo systemctl enable docker")
    stdout1, stderr1 = node.execute("sudo systemctl start docker")   

## Verify IP Addresses are configured

In [ ]:
slice = fablib.get_slice(slice_name)
slice.list_interfaces();

## Run iperf3 after performance tuning

Run iperf3 between every pair of the Nodes.

In [ ]:
slice = fablib.get_slice(slice_name)


source_name = f"{site1.lower()}-w1-a"
source_node = slice.get_node(name=source_name)

source_addr = source_node.get_interface(network_name=network_name).get_ip_addr()

node_names = [f"{site1.lower()}-w1-a", f"{site1.lower()}-w2-a", f"{site1.lower()}-w3-a"]

for i in range(len(node_names)):
    source_node = slice.get_node(name=node_names[i])
    source_addr = source_node.get_interface(network_name=network_name).get_ip_addr()
    
    for j in range(len(node_names)):
        if i==j:
            continue
        print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
        stdout1, stderr1 = source_node.execute("docker run -d --rm "
                                            "--network host "
                                            "fabrictestbed/slice-vm-rocky8-multitool:0.0.2 "
                                            "iperf3 -s -1"
                                            , quiet=True, output_file=f"{source_node.get_name()}.log");

        print(f"Source:  {node_names[i]} to Dest: {node_names[j]}")
        dest_node = slice.get_node(name=node_names[j])

        stdout2, stderr2 = dest_node.execute("docker run --rm "
                                            "--network host "
                                            "fabrictestbed/slice-vm-rocky8-multitool:0.0.2 "
                                            f"iperf3 -c {source_addr} -P 10 -t 30 -i 10 -O 10"
                                            , quiet=False, output_file=f"{dest_node.get_name()}.log");
        print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()